**For julia to run in google colab run this command**

This will take few minutes

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.4.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CuArrays"
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"'
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

Installing Julia 1.4.2 on the current Colab Runtime...
2020-07-11 06:40:34 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.4/julia-1.4.2-linux-x86_64.tar.gz [99093958/99093958] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
    Cloning default registries into `~/.julia`
    Cloning registry from "https://github.com/JuliaRegistries/General.git"
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed VersionParsing ── v1.2.0
  Installed MbedTLS_jll ───── v2.16.6+1
  Installed SoftGlobalScope ─ v1.0.10
  Installed ZeroMQ_jll ────── v4.3.2+4
  Installed Parsers ───────── v1.0.7
  Installed IJulia ────────── v1.21.2
  Installed Conda ─────────── v1.4.1
  Installed JSON ──────────── v0.21.0
  Installed ZMQ ───────────── v1.2.1
  Installed MbedTLS ───────── v1.0.2
######################################################################## 100.0%
#######################################################################

**Loading Packages**

In [1]:
using Pkg
Pkg.add("Flux")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
  Installed OpenSpecFun_jll ────────────── v0.5.3+3
  Installed CodecZlib ──────────────────── v0.7.0
  Installed DiffResults ────────────────── v1.0.2
  Installed ZipFile ────────────────────── v0.9.2
  Installed Tracker ────────────────────── v0.2.7
  Installed Juno ───────────────────────── v0.8.2
  Installed ForwardDiff ────────────────── v0.10.12
  Installed Flux ───────────────────────── v0.8.3
  Installed CompilerSupportLibraries_jll ─ v0.3.3+0
  Installed Media ──────────────────────── v0.5.0
  Installed SpecialFunctions ───────────── v0.10.3
  Installed CommonSubexpressions ───────── v0.3.0
  Installed DiffRules ──────────────────── v1.0.1
  Installed TranscodingStreams ─────────── v0.9.5
######################################################################### 100.0%
######################################################################### 100.0%
   Updating `~/.julia/environments/v1.4/Project.toml`
  [587475ba] + Flux v0.8.3
   Updating `~/.ju

In [2]:
using Plots,Statistics,Flux,Flux.Data.MNIST,Printf

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260
┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1260


In [3]:
using Base.Iterators: repeated, partition

In [4]:
using Flux: onehotbatch, onecold, crossentropy, throttle

In [5]:
Pkg.add("BSON")

  Resolving package versions...
  Installed BSON ─ v0.2.6
   Updating `~/.julia/environments/v1.4/Project.toml`
  [fbb218c0] + BSON v0.2.6
   Updating `~/.julia/environments/v1.4/Manifest.toml`
  [fbb218c0] + BSON v0.2.6


In [6]:
using  BSON

┌ Info: Precompiling BSON [fbb218c0-5317-5bc6-957e-2ee96dd4b1f0]
└ @ Base loading.jl:1260


**Loading Data**

we load data from julia package itself

In [7]:

train_label= MNIST.labels(:train)

┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /root/.julia/packages/Flux/qXNjB/src/data/mnist.jl:24
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /root/.julia/packages/Flux/qXNjB/src/data/mnist.jl:24
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /root/.julia/packages/Flux/qXNjB/src/data/mnist.jl:24
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /root/.julia/packages/Flux/qXNjB/src/data/mnist.jl:24


60000-element Array{Int64,1}:
 5
 0
 4
 1
 9
 2
 1
 3
 1
 4
 3
 5
 3
 ⋮
 7
 8
 9
 2
 9
 5
 1
 8
 3
 5
 6
 8

In [8]:
train_img = MNIST.images(:train)

60000-element Array{Array{Gray{FixedPointNumbers.Normed{UInt8,8}},2},1}:
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gr

In [9]:
test_label = MNIST.labels(:test)   # load labels

10000-element Array{Int64,1}:
 7
 2
 1
 0
 4
 1
 4
 9
 5
 9
 0
 6
 9
 ⋮
 5
 6
 7
 8
 9
 0
 1
 2
 3
 4
 5
 6

In [10]:

test_img =MNIST.images(:test) # load test labels

10000-element Array{Array{Gray{FixedPointNumbers.Normed{UInt8,8}},2},1}:
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gr

In [11]:
train_img[1]  # show image

**Model Building**

In [23]:
# building a simple model


m =Chain(           
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax)

model = m


Chain(Dense(784, 32, relu), Dense(32, 10), softmax)

In [24]:
X = hcat(float.(reshape.(train_img, :))...) #stack all the images
Y = onehotbatch(train_label, 0:9) # just a common way to encode categorical variables

10×60000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  1  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  1  0  1  0  0  0  0     0  0  0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  1  0  1     0  0  0  0  0  0  0  0  1  0  0  0
 0  0  1  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  1  0  …  0  0  0  0  0  1  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  1  0  0  0  0  0  1  0  0  0  1
 0  0  0  0  1  0  0  0  0  0  0  0  0     0  0  1  0  1  0  0  0  0  0  0  0

**Loss Function and Accuracy**

In [41]:
loss(x, y) = crossentropy(model(x), y) 
opt = ADAM();      # popular stochastic gradient descent variant

accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))     # cute way to find average of correct guesses

dataset = repeated((X,Y),200)       # repeat the data set 200 times, as opposed to @epochs 200 ...
evalcb = () -> @show(loss(X, Y))    # callback to show loss

#15 (generic function with 1 method)

In [49]:
 Flux.train!(loss, params(model), dataset, opt, cb = throttle(evalcb, 10)); #took me ~2 minutes to train on GPU

loss(X, Y) = 0.1288626f0 (tracked)
loss(X, Y) = 0.119610295f0 (tracked)
loss(X, Y) = 0.111465156f0 (tracked)
loss(X, Y) = 0.104163185f0 (tracked)
loss(X, Y) = 0.097341746f0 (tracked)
loss(X, Y) = 0.09151615f0 (tracked)
loss(X, Y) = 0.08603765f0 (tracked)
loss(X, Y) = 0.081104495f0 (tracked)
loss(X, Y) = 0.07682921f0 (tracked)
loss(X, Y) = 0.072693735f0 (tracked)


In [50]:
size(test_img)

(10000,)

In [52]:
Flux.Data.MNIST.images(:test)[5287] # give me a number 1-10000

In [53]:
# Same preprocessing
test_X = hcat(float.(reshape.(Flux.Data.MNIST.images(:test), :))...)
test_Y = onehotbatch(Flux.Data.MNIST.labels(:test), 0:9);

m(test_X[:,5287])       # Note the 7th index ( corresponding to the digit 6 ) is nearly 1

Tracked 10-element Array{Float32,1}:
 1.4339258f-8
 1.9824803f-8
 3.6036883f-7
 8.278516f-9
 2.1289566f-6
 7.799352f-8
 0.99999607f0
 1.1303876f-11
 1.2854244f-6
 7.762031f-11

In [54]:
onecold(m(test_X[:,5287])) - 1 

6

In [55]:
accuracy(X, Y)

0.9801666666666666

In [56]:
# Test set accuracy
accuracy(test_X, test_Y)

0.9625